## Fix symbol & Contruct big json file

In [1]:
import glob, json

In [2]:
# Remove "null" in json files
json_file = glob.glob(pathname='*/*.json')
for j_file in json_file:
    with open(j_file, 'r', encoding='utf-8') as f:
        content = json.load(f)
        # content = json.dumps(content).replace(u'u2019', "'")
    # content_2 = json.dumps(content).replace("’", "'")
    if 'answers' in content:
        content['answers'] = [x for x in content['answers'] if x is not None]
    with open(j_file, 'w', encoding='utf-8') as f:
        json.dump(content, fp=f, indent=4, ensure_ascii=False)
    # with open(j_file, 'r', encoding='utf-8') as f:
    #     data = f.read()
    #     data = data.replace(u'\u2019', "'")
    # with open(j_file, 'w') as f:
    #     f.write(data)
    print(j_file)
    # break


Json file for web setup/auto-driving-aligned.json
Json file for web setup/bitcoin-invest-aligned.json
Json file for web setup/Would-you-get-into-a-self-driving-car_0820_1.json
Json file for web setup/Should-I-invest-in-Bitcoin_0815_1.json


### Construct big Json file

In [3]:
import os

In [12]:
scenario = {
    "auto-driving" : {
        "file_name" : "car.json",
        "data_base_dir" : "claim center and statistics/automos driving",
        "web_base_dir": "Json file for web setup",
        "question": "Would you get into a self-driving car?",
        "url" : "https://www.quora.com/Would-you-get-into-a-self-driving-car",
    },
    "bitcoin-invest" : {
        "file_name" : "bitcoin.json",
        "data_base_dir": "claim center and statistics/bitcoin investment",
        "web_base_dir": "Json file for web setup",
        "question" : "Should I invest in Bitcoin?",
        "url" : "https://www.quora.com/Should-I-invest-in-Bitcoin",
    }
}


In [7]:
## Calculate and count in big Json file
def percentage_and_count(base_dir):
    stance_proportion_path = os.path.join(base_dir, 'stance_proportion.json')
    with open(stance_proportion_path, 'r') as f:
        stance_proportion = json.load(f)
    output = {
        "percentage" : {},
        "count" : stance_proportion,
    }
    claims_count = sum(stance_proportion.values())
    output["percentage"] = {
        k: "{:.0%}".format(v/claims_count) for k, v in stance_proportion.items()
    }
    return output

def create_claim_center_dict(base_dir):
    stance_count_path = os.path.join(base_dir, 'stance_count.json')
    supportiveness_path = os.path.join(base_dir, 'supportiveness.json')

    with open(stance_count_path, 'r') as f:
        stance_count = json.load(f)
    cs_to_st = {j:k for k,v in stance_count.items() for j in v}
    with open(supportiveness_path, 'r') as f:
        supportiveness = json.load(f)

    output = { "positive" : [], "neutral": [], "negative": [] }
    for t in supportiveness:
        st = cs_to_st[t['text']]
        output[st].append(t)
    return output

In [15]:
for sc, config in scenario.items():
    # get answer
    answer_data_path = os.path.join(config['web_base_dir'], f"{sc}-aligned.json")
    with open(answer_data_path, 'r') as f:
        answer_data = json.load(f)['answers']
    
    statistic_data = percentage_and_count(base_dir=config['data_base_dir'])
    claim_center_data = create_claim_center_dict(base_dir=config['data_base_dir'])
    # combine data
    big_json_data = {}
    big_json_data['question'] = config['question']
    big_json_data['source'] = config['url']
    big_json_data.update(statistic_data)
    big_json_data['claim_centers'] = claim_center_data
    big_json_data['answer'] = answer_data

    # save big json data
    big_json_data_path = os.path.join('big_json_data', config['file_name'])
    with open(big_json_data_path, 'w') as f:
        json.dump(big_json_data, fp=f, indent=4, ensure_ascii=False)


In [8]:
percentage_and_count(base_dir=scenario['auto-driving']['data_base_dir'])

{'percentage': {'neutral': '60%', 'positive': '20%', 'negative': '21%'},
 'count': {'neutral': 49, 'positive': 16, 'negative': 17}}

In [9]:
create_claim_center_dict(base_dir=scenario['auto-driving']['data_base_dir'])


{'positive': [{'text': 'What I mean is, autonomous cars would be great, IF THEY WERE THE ONLY CARS OUT THERE',
   'supportiveness': 0.07317073170731707},
  {'text': 'Yes, I will.', 'supportiveness': 0.06097560975609756},
  {'text': 'I think that this technology has the potential to be very safe and convenient',
   'supportiveness': 0.036585365853658534},
  {'text': 'the research shows people are amazingly trusting of them',
   'supportiveness': 0.024390243902439025}],
 'neutral': [{'text': 'Yes, I really would like to have a self-driving car',
   'supportiveness': 0.21951219512195122},
  {'text': 'Yes', 'supportiveness': 0.10975609756097561},
  {'text': 'Yes, you would.', 'supportiveness': 0.08536585365853659},
  {'text': 'Yes, definitely.', 'supportiveness': 0.08536585365853659},
  {'text': 'Yes, as soon as they work and are available at a reasonable price.',
   'supportiveness': 0.04878048780487805},
  {'text': 'Not at the time of answering.',
   'supportiveness': 0.04878048780487805